# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [54]:
# Import Clean City Data
weather = os.path.join('..', 'WheaterPy','Output_Data','Clean_City_Data.csv')
weather_df = pd.read_csv(weather)
weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Clyde River,70.4692,-68.5914,-0.87,86,40,7.72,CA,1622047223
1,Lebu,-37.6167,-73.6500,15.92,81,98,3.63,CL,1622047165
2,Busselton,-33.6500,115.3333,11.69,64,100,5.46,AU,1622047242
3,Rikitea,-23.1203,-134.9692,24.84,79,38,11.23,PF,1622047221
4,Alofi,-19.0595,-169.9187,23.94,94,88,1.54,NU,1622047344
...,...,...,...,...,...,...,...,...,...
562,Jaciara,-15.9653,-54.9683,33.28,39,6,0.73,BR,1622047966
563,Verkhnevilyuysk,63.4453,120.3167,6.32,88,91,1.69,RU,1622047966
564,Lahaina,20.8783,-156.6825,23.03,83,36,3.10,US,1622047967
565,Superior,46.7208,-92.1041,10.67,55,75,7.72,US,1622047967


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [55]:
# Store latitude and longitude in locations
world_location = weather_df[['Lat', 'Lng']]

# Store humidity
humidity = weather_df['Humidity (%)']

In [60]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(world_location, weights=humidity, 
                                 dissipating=False, max_intensity=1,
                                 point_radius=0.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:
fitting_weather = weather_df[(weather_df['Max Temp (C)']>=20) & (weather_df['Max Temp (C)']<=35) & (weather_df['Wind Speed (mph)'])<=25 &(weather_df['Cloudiness (%)']== 0,:]
fitting_weather                        

SyntaxError: invalid syntax (<ipython-input-62-b246f9faa53e>, line 1)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
